In [1]:
from pdb import run
from fff.simulation import run_calculator
from fff.simulation.utils import read_from_string, write_to_string
from _pytest.fixtures import fixture
import ase
from ase.build import molecule
from dataclasses import dataclass
from pathlib import Path
from typing  import Dict, Any, List, Optional
import os
import psutil
from functools import partial, update_wrapper
import json
import pickle
import time
import multiprocessing as mp
import matplotlib.pyplot as plt

import matplotlib.pyplot as plt
import numpy as np
import math

import sys
sys.path.append('../../multisite_')
# sys.path.append('../my_util')
from colmena.queue.evo_sch import *
from colmena.models import Result, ResourceRequirements
from my_util.data_structure import *

import logging
from IPython.display import display
logger = logging.getLogger()
logger.setLevel(logging.INFO)  # 设置日志级别为 INFO 或其他你想要的级别

class JupyterHandler(logging.Handler):
    def emit(self, record):
        display(self.format(record))

handler = JupyterHandler()
handler.setLevel(logging.INFO)  # 设置处理程序的日志级别
logger.addHandler(handler)

ModuleNotFoundError: No module named 'colmena.queue.evo_sch'

In [2]:
class test_queue:
    def __init__(self) -> None:
        self.available_resources = {"cpu":56, "gpu": 4, "memory": "128G"}
        self.result_list = {}
        self.methods = {'train', 'evaluate', 'run_calculator', 'run_sampling'}
        self._hist_data = historical_data(methods=self.methods, queue=self)
        available_tasks = {}
        for method in self.methods:
            available_tasks[method] = []
        self._available_tasks = available_task(available_tasks)
        self.evosch = evosch2(resources=self.available_resources, at = self._available_tasks, hist_data=self._hist_data)
        
        
        with open("/home/lizz_lab/cse12232433/project/colmena/multisite_/finetuning-surrogates/runs/hist_data/task_queue_audit.pkl",'rb')as f:
            task_queue_audit = pickle.load(f)
            
        # for to_run in task_queue_audit:
        for i in range(20):
            to_run = task_queue_audit[i]
            task_type = 'audit'
            atoms = to_run.atoms
            atoms.set_center_of_mass([0,0,0])
            xyz = write_to_string(atoms, 'xyz')
            input_args = (xyz,)
            input_kargs = {}
            result = Result(
                (input_args,input_kargs), 
                method='run_calculator', 
                topic='simulate',
                keep_inputs = True,
                serialization_method='pickle',
                task_info={'traj_id': to_run.traj_id, 'task_type': task_type,
                                           'ml_energy': to_run.ml_eng, 'xyz': xyz},
                resources=ResourceRequirements(cpu=8, gpu=0)
                
            )
            result.serialize()
            self.result_list[result.task_id] = result
            self._available_tasks.add_task_id(task_name="run_calculator", task_id=result.task_id)
            
tq = test_queue()
hist_path = [] 
hist_path.append("/home/lizz_lab/cse12232433/project/colmena/multisite_/finetuning-surrogates/runs/hist_data/simulation-results-20240319_230707.json")
hist_path.append("/home/lizz_lab/cse12232433/project/colmena/multisite_/finetuning-surrogates/runs/hist_data/inference-results-20240319_230707.json")
hist_path.append("/home/lizz_lab/cse12232433/project/colmena/multisite_/finetuning-surrogates/runs/hist_data/sampling-results-20240319_230707.json")
hist_path.append("/home/lizz_lab/cse12232433/project/colmena/multisite_/finetuning-surrogates/runs/hist_data/training-results-20240319_230707.json")

tq._hist_data.get_features_from_his_json(hist_path)

print(tq._hist_data.queue)
print(tq.evosch.get_task_nums())
print(tq.evosch.at.get_all())
tq.evosch.population = tq.evosch.generate_population(100)
predifine_ind = tq.evosch.population[-1]
print(predifine_ind.task_allocation)



None
{'run_calculator': ['7271c5f4-c8a5-4eeb-a565-ce1ff70ee197', '7d033b69-6974-4c93-b4d5-029bcfbacf89', 'e6b60572-5640-4afa-8b57-7e54e72d853e', '5f1bc92b-3e2b-4a1e-9d34-fd7255830efc', '5b098791-9c9f-469f-b270-d4c7bd05cc68', 'eab08b85-6fa8-440e-9d2c-9fe27b2234c4', '3dd43b50-9bb2-4fc1-88bc-c001b8129761', '030d66fc-76af-4657-a4d9-3ab121958991', 'e861b4ce-59ef-43d5-9c6b-7ca99f26b6d5', '12a2fac2-49e6-435b-b065-fa47626d1528', '7860597e-be33-4445-88ee-00797daccda6', '7a9464b8-bf98-440d-bdbb-25849fa64b95', 'cfa69881-1b95-44c8-aac1-60aee3a8ee01', '3ed46129-78a4-4522-ac2d-adbd3a30e0fe', '8000f8f2-31dd-433b-a180-b236eea45cd9', 'ca0239df-3870-41a4-8617-088292d84f1f', '1fe68e79-6181-41f4-8a74-51ed3c513966', '1c666824-a0ba-4929-a598-85303b29bb47', 'a5514fb2-6ac0-4e28-98b6-ed374c79755b', '33a5453f-c4ff-4576-9c27-04b1488c9859'], 'train': [], 'evaluate': [], 'run_sampling': []}
[{'name': 'run_calculator', 'task_id': 'cfa69881-1b95-44c8-aac1-60aee3a8ee01', 'resources': {'cpu': 8}}, {'name': 'run_calcul

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [3]:
## predifined resources score
tq.evosch.hist_data.random_forest_train()
tq.evosch.hist_data.estimate_batch([predifine_ind])
tq.evosch.fitness(predifine_ind)
print(predifine_ind.score)

'train:run_calculator model'

'method: run_calculator, random forest regressor score: 0.9581566679174408'

'train:train model'

'method: train, random forest regressor score: 0.9497878149701859'

'train:evaluate model'

'train:run_sampling model'

'method: run_sampling, random forest regressor score: 0.9154976126383643'

-480.28092885017395


In [4]:
best_ind = tq.evosch.run_ga(100)
print(best_ind)

'train:run_calculator model'

'method: run_calculator, random forest regressor score: 0.9581566679174408'

'train:train model'

'method: train, random forest regressor score: 0.9497878149701859'

'train:evaluate model'

...

'Generation 98: -358.4481761455536'

'Generation 99: -358.4481761455536'

'score of all ind:[-358.4481761455536, -358.4481761455536, -358.4481761455536, -358.4481761455536, -358.4481761455536, -358.4481761455536, -358.4481761455536, -358.4481761455536, -358.4481761455536, -358.4481761455536, -358.4481761455536, -358.4481761455536, -358.4481761455536, -358.4481761455536, -358.4481761455536, -358.4481761455536, -358.4481761455536, -358.4481761455536, -358.4481761455536, -358.4481761455536, -358.4481761455536, -358.4481761455536, -358.4481761455536, -358.4481761455536, -358.4481761455536, -358.4481761455536, -358.4481761455536, -358.4481761455536, -358.4481761455536, -358.4481761455536, -358.4481761455536, -358.4481761455536, -358.4481761455536, -358.4481761455536, -358.4481761455536, -358.4481761455536, -358.4481761455536, -358.4481761455536, -358.4481761455536, -358.4481761455536, -358.4481761455536, -358.4481761455536, -358.4481761455536, -358.4481761455536, -358.4481761455536, -358.4481761455536, -358.4481761455536, -358.4481761455536, -358.4481761455536, -

"Best ind:individual(individual_id=20204, tasks_nums={'run_calculator': 20, 'train': 0, 'evaluate': 0, 'run_sampling': 0}, total_resources={'cpu': 56, 'gpu': 4, 'memory': '128G'}, total_time=0, max_time=0, score=-358.4481761455536, predict_run_seq=[{'name': 'run_calculator', 'task_id': '12a2fac2-49e6-435b-b065-fa47626d1528', 'start_time': 1711040952.3539863, 'finish_time': 1711041310.8021624, 'total_runtime': 358.4481761455536, 'resources': {'cpu': 1}}, {'name': 'run_calculator', 'task_id': '5f1bc92b-3e2b-4a1e-9d34-fd7255830efc', 'start_time': 1711040952.3539863, 'finish_time': 1711041195.7127078, 'total_runtime': 243.35872149467468, 'resources': {'cpu': 6}}, {'name': 'run_calculator', 'task_id': '030d66fc-76af-4657-a4d9-3ab121958991', 'start_time': 1711040952.3539863, 'finish_time': 1711041245.2627015, 'total_runtime': 292.9087152481079, 'resources': {'cpu': 5}}, {'name': 'run_calculator', 'task_id': '7860597e-be33-4445-88ee-00797daccda6', 'start_time': 1711040952.3539863, 'finish_tim

individual(individual_id=20204, tasks_nums={'run_calculator': 20, 'train': 0, 'evaluate': 0, 'run_sampling': 0}, total_resources={'cpu': 56, 'gpu': 4, 'memory': '128G'}, total_time=0, max_time=0, score=-358.4481761455536, predict_run_seq=[{'name': 'run_calculator', 'task_id': '12a2fac2-49e6-435b-b065-fa47626d1528', 'start_time': 1711040952.3539863, 'finish_time': 1711041310.8021624, 'total_runtime': 358.4481761455536, 'resources': {'cpu': 1}}, {'name': 'run_calculator', 'task_id': '5f1bc92b-3e2b-4a1e-9d34-fd7255830efc', 'start_time': 1711040952.3539863, 'finish_time': 1711041195.7127078, 'total_runtime': 243.35872149467468, 'resources': {'cpu': 6}}, {'name': 'run_calculator', 'task_id': '030d66fc-76af-4657-a4d9-3ab121958991', 'start_time': 1711040952.3539863, 'finish_time': 1711041245.2627015, 'total_runtime': 292.9087152481079, 'resources': {'cpu': 5}}, {'name': 'run_calculator', 'task_id': '7860597e-be33-4445-88ee-00797daccda6', 'start_time': 1711040952.3539863, 'finish_time': 171104

In [5]:
task_allocation = best_ind.task_allocation
task_allocation.sort(key=lambda x:x['total_runtime'], reverse=True)
cpus = 0
for task in task_allocation:
    cpus += task['resources']['cpu']
    print(task["resources"],task["total_runtime"])
    
print(f"cost toatl cpu:{cpus}")



{'cpu': 1} 358.4481760846666
{'cpu': 2} 299.5212459472044
{'cpu': 5} 292.9087151354445
{'cpu': 1} 285.2688049357655
{'cpu': 1} 285.2688049357655
{'cpu': 3} 272.756118617515
{'cpu': 4} 272.70129784416423
{'cpu': 1} 270.3268304230463
{'cpu': 1} 270.3268304230463
{'cpu': 1} 270.3268304230463
{'cpu': 1} 260.2303486073963
{'cpu': 1} 260.2303486073963
{'cpu': 6} 243.35872156638055
{'cpu': 7} 209.20008061965964
{'cpu': 7} 209.20008061965964
{'cpu': 6} 167.66169149612543
{'cpu': 6} 166.34229022388973
{'cpu': 7} 154.72511513281816
{'cpu': 1} 143.1057245972799
{'cpu': 1} 136.14318593261822
cost toatl cpu:63


In [21]:
task_allocation = sorted(predifine_ind.predict_run_seq, key=lambda x:x['total_runtime'], reverse=True)
print(type(predifine_ind.predict_run_seq.sort(key=lambda x:x['total_runtime'], reverse=True)))

<class 'NoneType'>


In [9]:
print(8%2)

0
